# Artificial intelligence models built are:
 ### 1. cnn( with various layers)
 ### 2. lstm(with various layers and optimizers)
 ### 3. embedding values of cnn taken into svm 

In [3]:
from __future__ import print_function
import numpy as np

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Embedding
from keras.layers import GlobalAveragePooling1D
from keras.datasets import imdb

import pandas as pd     # for reading data operations
import keras

from keras.preprocessing.text import Tokenizer          # for tokenizing text
from keras.preprocessing.sequence import pad_sequences  # for padding sentences with zeros. To make the sentence length same
from keras.utils import to_categorical                  # for one-hot encoding of the labels
from keras.layers import Dense, Input, Flatten, Dropout, BatchNormalization
from keras.layers import SimpleRNN, LSTM, GRU, Input, Concatenate
from keras.layers import Conv1D, MaxPooling1D, GlobalMaxPool1D, Embedding, GlobalAvgPool1D
from keras.models import Sequential, Model

C:\Users\Dr. Jhansi Rani.SSV_BALARAM_MOH\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
C:\Users\Dr. Jhansi Rani.SSV_BALARAM_MOH\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Dr. Jhansi Rani.SSV_BALARAM_MOH\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8

In [1]:
import pickle
filename = '4000reviews_preprocessed.csv'
infile = open(filename,'rb')
review3000_dt = pickle.load(infile)
infile.close()

In [2]:
import pickle
filename = 'final_preprocessed.csv'
infile = open(filename,'rb')
final = pickle.load(infile)
infile.close()

In [4]:
review3000_dt.sample()

,review,sentiment,review_processed_1
234,I wish they kept the same cast...probably woul...,0,wish kept cast probably would cost would worth


In [5]:
pd.set_option('display.max_colwidth',-1)


In [6]:
# review3000_dt['sentiment'] = review3000_dt['sentiment'].replace({0:1, 1:0})


In [7]:
# review3000_r_dt = review3000_dt.drop(labels=['review_processed_1'], axis=1)

In [8]:
# review3000_rp1_dt = review3000_dt.drop(labels=['review'], axis=1)

In [9]:
from sklearn.model_selection import train_test_split
X_train_rp1, X_test_rp1, y_train_rp1, y_test_rp1 = train_test_split(review3000_dt['review_processed_1'],review3000_dt['sentiment'],test_size=0.20,random_state=123)

In [10]:
from sklearn.model_selection import train_test_split
X_train_r, X_test_r, y_train_r, y_test_r = train_test_split(review3000_dt['review'],review3000_dt['sentiment'],test_size=0.20,random_state=123)

In [11]:
review3000_dt.review_processed_1[2000]

'original much better music performance seemed lacking heart seemed like flat also felt like humor lightheartedness original missing opinion'

In [12]:
# Embedding
max_features = 20000
maxlen = 5000
embedding_size = 128

# Convolution
kernel_size = 5
filters = 64
pool_size = 4

# LSTM
lstm_output_size = 70

# Training
batch_size = 30
epochs = 4


In [13]:
## Define the sequence lengths, max number of words and embedding dimensions
MAX_SEQUENCE_LENGTH = 5000   # Sequence length of each sentence. If more, crop. If less, pad with zeros
MAX_NB_WORDS = 10000       # Top 20000 frequently occuring words

In [14]:
tokenizer = Tokenizer(num_words=MAX_NB_WORDS)   # get the frequently occuring words
tokenizer.fit_on_texts(X_train_rp1)           
train_sequences = tokenizer.texts_to_sequences(X_train_rp1)
test_sequences = tokenizer.texts_to_sequences(X_test_rp1)

word_index = tokenizer.word_index               # dictionary containing words and their index
# print(tokenizer.word_index)                   # print to check
print('Found %s unique tokens.' % len(word_index)) # total words in the corpus
X_train_dt = pad_sequences(train_sequences, maxlen=MAX_SEQUENCE_LENGTH) # get only the top frequent words on train
X_test_dt = pad_sequences(test_sequences, maxlen=MAX_SEQUENCE_LENGTH)   # get only the top frequent words on test

print(X_train_dt.shape)
print(X_test_dt.shape)

Found 3578 unique tokens.
(2400, 5000)
(600, 5000)


In [15]:
tokenizer = Tokenizer(num_words=MAX_NB_WORDS)   # get the frequently occuring words
tokenizer.fit_on_texts(X_train_r)           
train_sequences = tokenizer.texts_to_sequences(X_train_r)
test_sequences = tokenizer.texts_to_sequences(X_test_r)
final_seq = tokenizer.texts_to_sequences(final.Review)
word_index = tokenizer.word_index               # dictionary containing words and their index
# print(tokenizer.word_index)                   # print to check
print('Found %s unique tokens.' % len(word_index)) # total words in the corpus
X_train_r_dt = pad_sequences(train_sequences, maxlen=MAX_SEQUENCE_LENGTH) # get only the top frequent words on train
X_test_r_dt = pad_sequences(test_sequences, maxlen=MAX_SEQUENCE_LENGTH)   # get only the top frequent words on test
final_dt = pad_sequences(final_seq,maxlen=MAX_SEQUENCE_LENGTH)
print(X_train_r_dt.shape)
print(X_test_r_dt.shape)

Found 4587 unique tokens.
(2400, 5000)
(600, 5000)


In [16]:
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding
from keras.layers import Conv1D, GlobalMaxPooling1D

In [17]:
print('Build model...')

model = Sequential()
model.add(Embedding(max_features, embedding_size, input_length=maxlen))
model.add(Dropout(0.25))
model.add(Conv1D(filters,
                 kernel_size,
                 padding='valid',
                 activation='relu',
                 strides=1))
model.add(MaxPooling1D(pool_size=pool_size))
model.add(LSTM(lstm_output_size))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

print('Train...')
model.fit(X_train_r_dt, y_train_r,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(X_test_r_dt, y_test_r))
score, acc = model.evaluate(X_test_r_dt, y_test_r, batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)

Build model...
Train...
Train on 2400 samples, validate on 600 samples
Epoch 1/4
2400/2400 [==============================] - 425s 177ms/step - loss: 0.5993 - acc: 0.6800 - val_loss: 0.5001 - val_acc: 0.7433
Epoch 2/4
2400/2400 [==============================] - 376s 157ms/step - loss: 0.3619 - acc: 0.8471 - val_loss: 0.4282 - val_acc: 0.8183
Epoch 3/4
2400/2400 [==============================] - 361s 150ms/step - loss: 0.1995 - acc: 0.9308 - val_loss: 0.4854 - val_acc: 0.8183
Epoch 4/4
600/600 [==============================] - 21s 35ms/step
Test score: 0.544458407908678
Test accuracy: 0.8066666632890701


In [18]:
pred_train = model.predict_classes(X_train_r_dt)
pred_test = model.predict_classes(X_test_r_dt)
from sklearn.metrics import f1_score

f1_train=f1_score( y_train_r, pred_train, labels=None, pos_label=1, average='binary')
f1_test=f1_score( y_test_r, pred_test, labels=None, pos_label=1, average='binary')
print(f1_train)
print(f1_test)

0.9707943925233644
0.7251184834123222


In [32]:
final['sentiment']=final_predict_m5

f11=final.drop(labels=['Review','review_processed_1'], axis=1)

f11.sample(20)

f11.to_csv("model32.csv",header=True,index=False)

In [34]:
f11.sentiment.value_counts()

0    871
1    329
Name: sentiment, dtype: int64